In [1]:
%run 'naoqi/naoqi.py'
%run 'naoqi/almath.py'

In [2]:
import sys
import motion
import time
import math
import numpy as np

In [3]:
robotIP = "169.254.226.148" # A changer selon les cas

In [106]:
robotIP = "169.254.210.108"

In [4]:
robotIP = "127.0.0.1"

In [4]:
# Ouvre les connexions
motionProxy = ALProxy("ALMotion", robotIP, 9559) 
postureProxy = ALProxy("ALRobotPosture", robotIP, 9559)

In [5]:
def StiffnessOn(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "Body"
  pStiffnessLists = 1.0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)

def StiffnessOff(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "Body"
  pStiffnessLists = 0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)

def StiffnessOnArm(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "LArm"
  pStiffnessLists = 1.0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)

def StiffnessOffArm(proxy):
  #We use the "Body" name to signify the collection of all joints
  pNames = "LArm"
  pStiffnessLists = 0
  pTimeLists = 1.0
  proxy.stiffnessInterpolation(pNames, pStiffnessLists, pTimeLists)


In [22]:
StiffnessOff(motionProxy)

In [6]:
StiffnessOn(motionProxy)

In [112]:
postureProxy.goToPosture("StandZero", 0.5)

True

In [49]:
StiffnessOffArm(motionProxy)

In [50]:
StiffnessOnArm(motionProxy)

In [32]:
p = motionProxy.getPosition("LArm",0,True)


In [35]:
p[3:]=[0,0,0]
p
motionProxy.positionInterpolation("LArm", 0,
            path,axisMask, times, isAbsolute)

[0.16392914950847626, 0.0388508178293705, 0.03919321298599243, 0, 0, 0]

In [194]:
def cartesian_control():
    # Setting posture StandInit before hand is mandatory
    effector   = "LArm"
    space      = motion.FRAME_ROBOT
    path       = [ # Chemin représenté comme un ligne de points
     [0.0, +0.025, +0.00, 0.0, 0.0, 0.0],        # point 1
     [0.025, +0.0, +0.00, 0.0, 0.0, 0.0],        # point 2
     [0.0, -0.025, +0.00, 0.0, 0.0, 0.0],        # point 3
     [-0.025, +0.0, -0.00, 0.0, 0.0, 0.0]]#,        # point 4
   #  [0.0, -0.05, +0.00, 0.0, 0.0, 0.0],        # point 5
   #  [0.0, +0.05, +0.00, 0.0, 0.0, 0.0]] 
    axisMask = 7
    # Moment auxquel on doit passer aux points considérés
    times = [1, 2.0, 3.0, 4.0]#, 4.0, 4.5] # seconds
    isAbsolute = False
    
    motionProxy.positionInterpolation(effector, space,
            path,axisMask, times, isAbsolute)

In [ ]:
cartesian_control()

In [16]:
import pickle
with open("config_init.pickle","rb") as f:
    q = pickle.load(f)

names = ['LShoulderPitch', 'LShoulderRoll', 'LElbowYaw', 'LElbowRoll', 'LWristYaw', 'LHand']

q = [float(q[15])] + [float(q[16])] + [float(q[17])] + [float(q[18])] + [float(q[19])] + [float(q[32])]
print(q)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
pi = math.pi

def radian(a):
    return a * pi / 180

def angles():
    # Exemples de positionnement avec les angles
    motionProxy.setStiffnesses("LArm", 1.0)

    # Example showing how to set angles, using a fraction of max speed
    #names  = ["LShoulderPitch", "LElbowYaw"]
    angles  = q
    fractionMaxSpeed  = 0.1
    motionProxy.setAngles(names, angles, fractionMaxSpeed)

    time.sleep(3.0)
    #motionProxy.setStiffnesses("LArm", 0.0)


In [18]:
angles()

In [197]:
def angle2():
    # Example showing multiple trajectories
    names  = ["LShoulderPitch","LElbowRoll","LElbowYaw"]
    # Each joint can have lists of different lengths, but the number of
    # angles and the number of times must be the same for each joint.
    angleLists  = [ [30*TO_RAD,0],
                    [-50.0*TO_RAD, 0.0],
                   [-30.0*TO_RAD, 30.0*TO_RAD, 0.0]]
    timeLists   = [[1.0,6.0],[2.0, 3.0], [ 2.0, 3.0, 4.0]]
    isAbsolute  = True
    motionProxy.angleInterpolation(names, angleLists, timeLists, isAbsolute)
    

In [198]:
angle2()

In [21]:
p1 = motionProxy.getAngles(["LShoulderPitch", "LShoulderRoll", "LElbowYaw", "LElbowRoll", "LWristYaw", "LHand"],True)

In [22]:
p2 = motionProxy.getAngles(["LShoulderPitch", "LShoulderRoll", "LElbowYaw", "LElbowRoll", "LWristYaw", "LHand"],True)

In [23]:
p3 = motionProxy.getAngles(["LShoulderPitch", "LShoulderRoll", "LElbowYaw", "LElbowRoll", "LWristYaw", "LHand"],True)

In [24]:
config = np.vstack((p1,p2,p3))

In [25]:
config.shape


(3, 6)

In [26]:
config = config.T.reshape((6,3))

In [27]:
config.tolist()


[[-0.1181599572300911, -0.10281995683908463, -0.01691596210002899],
 [0.5307220220565796, 0.5307220220565796, 0.25920403003692627],
 [0.050580039620399475, 0.013764038681983948, -0.05373196303844452],
 [-1.4894720315933228, -1.2501680850982666, -1.251702070236206],
 [-1.5754599571228027, -1.507964015007019, -1.4910900592803955],
 [1.0, 1.0, 1.0]]